# Imports para Spark

In [47]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
from time import time

# Enviar os dados para o HDFS

os arquivos .csv devem estar dentro da pasta input do projeto.

In [ ]:
!hdfs dfs -mkdir /datasus

In [ ]:
!hdfs dfs -copyFromLocal /input/*.csv /datasus/

In [43]:
!hdfs dfs -ls /datasus

Found 4 items
-rw-r--r--   2 root supergroup   62492959 2021-07-16 20:15 /datasus/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   2 root supergroup   76520681 2021-07-16 20:15 /datasus/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   2 root supergroup   91120916 2021-07-16 20:15 /datasus/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   2 root supergroup    3046774 2021-07-16 20:15 /datasus/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


# Otimizar todos os dados do HDFS para uma tabela hive.

In [ ]:
dataframe_hdfs = spark.read.csv("/datasus/", header="true", sep=";", quote="\'", inferSchema=True)

In [ ]:
dataframe_hdfs.createOrReplaceTempView("covid19Temp")

In [ ]:
sqlContext.sql("create table covid19 as select * from covid19Temp");

In [ ]:
spark.sql("drop table covid19Temp")

In [42]:
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|             covid19|      false|
| default|primeiravisualizacao|      false|
+--------+--------------------+-----------+



In [ ]:
data_processamento = "2021-07-06"
data_referencia    = "2021-07-05"

# Primeira Visualização

In [33]:
# Casos Recuperados, Em acompanhamento

x = datetime.now()

Casos_Recuperados_DF = \
    sqlContext.sql( \
        "SELECT \
            FORMAT_NUMBER(SUM(Recuperadosnovos),0) AS Casos_Recuperados, \
            FORMAT_NUMBER(SUM(emAcompanhamentoNovos),0) AS Em_Acompanhamento \
        FROM \
            covid19 \
        WHERE \
            data = '" + data_processamento + "'").show()

y = datetime.now()
print("TEMPO DE EXECUÇÃO -->", y-x)

+-----------------+-----------------+
|Casos_Recuperados|Em_Acompanhamento|
+-----------------+-----------------+
|       17,262,646|        1,065,477|
+-----------------+-----------------+

TEMPO DE EXECUÇÃO --> 0:00:02.335001


# Segunda Visualização

In [39]:
# Casos confirmados

Casos_Confirmados_DF = \
    spark.sql( \
        "SELECT \
            FORMAT_NUMBER((SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + SUM(obitosAcumulado)),0) AS Acumulado, \
            (\
                SELECT \
                    FORMAT_NUMBER(SUM(casosNovos),0) \
                FROM \
                    covid19 \
                WHERE \
                    data = '" + data_referencia + "' \
             ) AS Casos_Novos, \
            FORMAT_NUMBER(((SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + SUM(obitosAcumulado)) * 100000) / SUM(populacaoTCU2019), 0) as Incidencia \
        FROM \
            covid19 \
        WHERE \
            data = '" + data_processamento + "'")

In [40]:
Casos_Confirmados_DF.show()

+----------+-----------+----------+
| Acumulado|Casos_Novos|Incidencia|
+----------+-----------+----------+
|19,908,799|     68,109|     3,158|
+----------+-----------+----------+



# Terceira Visualização

In [51]:
# Óbitos Confirmados

Casos_Obitos_DF = \
    spark.sql( \
        "SELECT \
            FORMAT_NUMBER(((SUM(obitosAcumulado))),0) AS Obitos_acumulados, \
            (\
                SELECT \
                    FORMAT_NUMBER(SUM(obitosNovos),0) \
                FROM \
                    covid19 \
                WHERE \
                    data = '" + data_referencia + "' \
            ) AS Obitos_novos, \
            ROUND(((SUM(obitosAcumulado)) * 100) / (SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + (SUM(obitosAcumulado))),2) AS Letalidade, \
            ROUND(((((SUM(obitosAcumulado))) * 100000)) / SUM(populacaoTCU2019), 2) AS Mortalidade \
        FROM \
            covid19 \
        WHERE data = '" + data_processamento + "'")

In [60]:
Casos_Obitos_DF.show()

# Salvar primeira visualização em uma tabela Hive

In [34]:
sqlContext.sql("CREATE TABLE primeiraVisualizacao as \
        SELECT \
            FORMAT_NUMBER(SUM(Recuperadosnovos),0) AS Casos_Recuperados, \
            FORMAT_NUMBER(SUM(emAcompanhamentoNovos),0) AS Em_Acompanhamento \
        FROM \
            covid19 \
        WHERE \
            data = '" + data_processamento + "'");

# Salvar segunda visualização em formato parquet e compressão snappy

In [41]:
Casos_Confirmados_DF.write.mode("overwrite").option("compression", "snappy").parquet("/user/datacovid/")

In [46]:
!hdfs dfs -ls /user/datacovid

Found 2 items
-rw-r--r--   2 root supergroup          0 2021-07-16 23:38 /user/datacovid/_SUCCESS
-rw-r--r--   2 root supergroup        961 2021-07-16 23:38 /user/datacovid/part-00000-84a9a882-d4e0-4b35-a793-4f19211a13e8-c000.snappy.parquet


# Salvar terceira visualização como um tópico no Kafka

In [78]:
Casos_Obitos_DF\
    .selectExpr("'terceira' AS key", "to_json(struct(*)) AS value") \
    .write\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka:9092")\
    .option("topic", "terceira-visualizacao")\
    .save()

In [83]:
prova = spark.read.\
    format("kafka").\
    option("kafka.bootstrap.servers", "kafka:9092").\
    option("subscribe", "terceira-visualizacao").load()

In [84]:
prova.show()

+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+
|                 key|               value|               topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+
|[74 65 72 63 65 6...|[7B 22 4F 62 69 7...|terceira-visualiz...|        0|     0|2021-07-17 00:26:...|            0|
|[74 65 72 63 65 6...|[7B 22 4F 62 69 7...|terceira-visualiz...|        0|     1|2021-07-17 00:26:...|            0|
+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+

